In [16]:
from Bio import SeqIO

FILE_PATH = '/home/gamran/genome_analysis/Warrior/genome/'
P_FILE_NAME = 'DK_0911_v03_p_ctg.fa'
H_FILE_NAME = 'DK_0911_v03_h_ctg.fa'

H_CTG_FILE_LOC = FILE_PATH + H_FILE_NAME
P_CTG_FILE_LOC = FILE_PATH + P_FILE_NAME

htgs = [h for h in SeqIO.parse(H_CTG_FILE_LOC, 'fasta')]
pCtgs = [p for p in SeqIO.parse(P_CTG_FILE_LOC, 'fasta')]

sumHtgs = 0
sumPCtgs = 0
for htg in htgs:
    # print('ID: %s length %i' %(htg.id, len(htg.seq)))
    sumHtgs += len(htg.seq)
for pCtg in pCtgs:
    # print('ID: %s length %i' %(pCtg.id, len(pCtg.seq)))
    sumPCtgs += len(pCtg.seq)
print('\
Total number of haplotigs: %i\n\
Total haplotig length: %i\n\
Total number of primary contigs: %i\n\
Total primary contig length: %i'\
%(len(htgs), sumHtgs, len(pCtgs), sumPCtgs))

Total number of haplotigs: 1176
Total haplotig length: 52126201
Total number of primary contigs: 94
Total primary contig length: 74427617
